Team Members
* Gaurav Karwa - R11889434
* Harshitha Arava - R11901126
* Shweta Sumant Koyande - R11904525
* Surendranath Avvaru - R11894279

The below notebook is checking transferability of our attack on phi 3.5 mini model

The following code is used to install the model. Replace the `HF_TOKEN` placeholder with your actual Hugging Face token.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login

# Your hf token goes here
login(token='HF_TOKEN')
torch.use_deterministic_algorithms(False)

# Define the model ID for the pre-trained model that you want to use
# Here, we are using the "microsoft/Phi-3.5-mini-instruct" model as an example
model_id = "microsoft/Phi-3.5-mini-instruct"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_id)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]


We are utilizing adversarial strings generated from an attack on the Llama 3.2 model. You can find the corresponding file in the folder. Simply upload it and execute the process.

In [ ]:
import pandas as pd

# Path to the CSV file containing the adversarial strings (generated from attack on Llama 3.2 model)
parquet_file_path = "/content/transferability_input_file"
df = pd.read_csv(parquet_file_path)

# Prepare inputs for inference: concatenate `best_string` and `positive_prompt` into a single 'input' column
# The 'input' column will be used to feed into the model for inference
df["input"] = df["best_string"] + " " + df["positive_prompt"]




The following cell performs inference on all the adversarial inputs and saves the results to a CSV file.

In [ ]:
results = []

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
     # Extract the input text from the current row
    input_text = row["input"]

    # Tokenize the input text
    inputs_encoded = tokenizer([input_text], return_tensors="pt").to("cuda")

    # Perform inference to generate a response
    with torch.no_grad():
        output = model.generate(**inputs_encoded, max_new_tokens=64, use_cache=True)

     # Decode the generated response from token IDs to text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Append the adversarial prompt and generated response to the results list
    results.append({
        "adverserial_prompt": row["input"],

        "reply": generated_text,
    })

# Save the results to a new CSV file
results_df = pd.DataFrame(results)
results_csv_file_path = "inference_results.csv"
results_df.to_csv(results_csv_file_path, index=False)

print(f"Inference complete. Results saved to '{results_csv_file_path}'.")


Inference complete. Results saved to 'inference_results.csv'.


End of transferability